In [325]:
import pandas as pd
import re
import numpy as np
import os

In [300]:
df = pd.read_csv('../data/raw-data/googleplaystore.csv')

## Data Exploration


In [301]:
df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [302]:
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [303]:
df.nunique()

App               9660
Category            34
Rating              40
Reviews           6002
Size               462
Installs            22
Type                 3
Price               93
Content Rating       6
Genres             120
Last Updated      1378
Current Ver       2832
Android Ver         33
dtype: int64

## Data Cleaning and transforming

#### Remove duplicates

In [304]:
df[df.duplicated(keep='first')]
df = df.sort_values(by=['App', 'Reviews'], ascending=[True, False])
df = df.drop_duplicates(subset=['App'], keep='first')

#### Reviews

* while preprocessing the Review column, we found  that there text values in the column

* by viewing this we can say that the row has shifted value and cannot be used, so we will drop it

In [305]:

df.reset_index(inplace=True)
df[df['Reviews'] == '3.0M']

,index,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
5806,10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [306]:
df = df.drop(5806, axis=0)

In [307]:
df['Reviews'] = df['Reviews'].apply(lambda x: int(x))

#### Installs

* Remove "+" & "," and convert to int type

In [308]:
df['Installs'] = df['Installs'].apply(lambda x: int(x.replace('+', '').replace(',', '')))

### Size

* Remove "M" & "K" and convert KB size to MB size

In [309]:
df['Size'] = df['Size'].apply(lambda x: str(x).replace('M', '').replace(',', ''))

In [310]:
df['Size'] = df['Size'].apply(lambda x: float(str(x).replace('k', '')) / 1000 if 'k' in str(x) else x)

* while preprocessing the Size column, we found that there are some values with 'Varies with device' which is not a valid value, so will convert them into NAN 

In [311]:
df['Size'] = df['Size'].apply(lambda x: str(x).replace('Varies with device', '0') if 'Varies with device' in str(x) else x)

In [312]:
df['Size'] = df['Size'].apply(lambda x: float(x) * 1_000_000)

* convert 'Varies with device' to NaN

In [313]:
df['Size'].describe()

count    9.659000e+03
mean     1.780249e+07
std      2.149347e+07
min      0.000000e+00
25%      2.900000e+06
50%      9.100000e+06
75%      2.500000e+07
max      1.000000e+08
Name: Size, dtype: float64

In [314]:
df['Size'].isnull().sum()

0

### Price

* Remove "$" sign and convert to float

In [315]:
df['Price'] = df['Price'].apply(lambda x: str(x).replace('$', '') if '$' in str(x) else str(x))
df['Price'] = df['Price'].apply(lambda x: float(x))

In [316]:
# df['Price'].isnull().sum()
df['Price'].describe()

count    9659.000000
mean        1.099299
std        16.852152
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       400.000000
Name: Price, dtype: float64

### Dates

In [317]:
df['Last Updated'] = pd.to_datetime(df['Last Updated'])
df["year"] = df["Last Updated"].dt.year
# df["month"] = df["Last Updated"].dt.month

In [318]:
df['year'].min(), df['year'].max()  

(2010, 2018)

## Extract processed dataset

In [326]:
output_directory = '../data/processed-data/'

year_ranges = [
    (2010, 2016),
    (2010, 2017),
    (2010, 2018)
]

split_dfs = {}

# create dataframe for each year range
for start_year, end_year in year_ranges:
    mask = (df['year'] >= start_year) & (df['year'] <= end_year)
    split_df = df.loc[mask]
    split_dfs[f'version_{start_year}_{end_year}'] = split_df

# export each split to a csv file
for key, split_df in split_dfs.items():
    file_name = f'{key}.csv'
    file_path = os.path.join(output_directory, file_name)
    split_df.to_csv(file_path, index=False)

    # print(f'Saved {key} to {file_name}')

# check number of records in splits
for key, split_df in split_dfs.items():
    print(f'{key}: {split_df.shape[0]} records')


version_2010_2016: 1581 records
version_2010_2017: 3374 records
version_2010_2018: 9659 records


## Transform

### Number of apps and average rating by Category 


In [320]:
# groups = df.groupby('Category').filter(lambda x: len(x) > 300).reset_index()

# # calculate the average rating
# average_rating = np.nanmean(groups['Rating'])

# # create a list of categories
# categories = list(set(groups['Category']))

# #prepare data for each category
# category_data = []
# for category in categories:
#     category_dict = {
#         'Category': category,
#         'Average_Rating': np.nanmean(groups.loc[groups['Category'] == category]['Rating']),
#         'Number_of_Apps': len(groups.loc[groups['Category'] == category]),
#     }
#     category_data.append(category_dict)


# df_category_summary = pd.DataFrame(category_data)

# df_category_summary


## Creating dummy variables

In [321]:
# categories = pd.get_dummies(df['Category'], prefix='Category', drop_first=True)
# types = pd.get_dummies(df['Type'], prefix='Type', drop_first=True)
# content_rating = pd.get_dummies(df['Content Rating'], prefix='Content_Rating', drop_first=True)
# genres = pd.get_dummies(df['Genres'], prefix='Genre', drop_first=True)
# frames = [df, categories, types, content_rating, genres]
# df = pd.concat(frames, axis=1)
# df = df.drop(['Category', 'Type', 'Content Rating', 'Genres', 'Installs', 'Current Ver', 'Android Ver'], axis=1)

In [322]:
# df.sample(5)